In [55]:
import os
import pandas as pd
import re

In [56]:
target = "NepQuake15/RawNepQuake15/"
file_list = [os.path.join(target,x) for x in os.listdir(target)]
file_list.sort()
file_list

['NepQuake15/RawNepQuake15/Nepali-Tweets-04-25.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-26.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-27.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-28.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-29.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-04-30.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-01.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-02.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-03.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-04.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-11.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-12.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-13.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-14.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-15.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-16.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-17.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-18.csv',
 'NepQuake15/RawNepQuake15/Nepali-Tweets-05-19

In [57]:
#### CLEANING ##### remove @usernames and external-urls
def cleanText(pdf):
    pdf['Text'] = pdf.apply(lambda x: re.sub('@[a-zA-Z]+','',x['Text']).strip(),axis=1) # remove @usernames
    pdf['Text'] = pdf.apply(lambda x: re.sub('#[a-zA-Z]+','',x['Text']).strip(),axis=1) # remove hash-tag
    pdf['Text'] = pdf.apply(lambda x: re.sub('\w+:\/\/[\w.\/]+',' ',x['Text']).strip(),axis=1)  # remove external urls
    pdf['Text'] = pdf.apply(lambda x: re.sub('[a-zA-Z]+','',x['Text']).strip(),axis=1)  # remove english alphabets
    pdf['Text'] = pdf.apply(lambda x: re.sub("\n+","\n",re.sub('( +\n +)+','\n',x['Text'])).strip(),axis=1) # remove new line feeds and extra spaces
    pdf['Text'] = pdf.apply(lambda x: re.sub("^\s-\s*|\s*-\s*$",'',x['Text']).strip(),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\.+",'.',x['Text']).strip(),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub(",+",',',x['Text']).strip(),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\?+",'\?',x['Text']).strip(),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("[।]+",'।',x['Text']).strip(),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("[\"]+",'\"',x['Text']).strip(),axis=1)
    return pdf


In [58]:
agg = pd.DataFrame()

In [59]:
for file in file_list:
    print(file)
    pdf = pd.read_csv(file)
    print('before',pdf.shape)
    pdf = cleanText(pdf)
    
    pdf.index = range(pdf.shape[0])
    
    ### Checking for shortest string in the dataframe
    lengths = pdf.apply(lambda x: len(x['Text'].split(" ")),axis=1)
    print("Min",min(lengths),"Max", max(lengths),"Avg",sum(lengths)/lengths.shape[0])
    
    ##### CLEANING ##### remove short sentences ----
    indexes = pdf.apply(lambda x: len(x['Text'].split(" "))>=5,axis=1)
    pdf = pdf[indexes]
    
    ### Drop Duplicates
    pdf = pdf.drop_duplicates()
    print("after",pdf.shape)
    
    if not agg.empty:
        agg = pd.concat([agg,pdf])
    else:
        agg = pdf
    
agg = agg.drop_duplicates()

NepQuake15/RawNepQuake15/Nepali-Tweets-04-25.csv
before (20500, 2)
Min 1 Max 67 Avg 11.25190243902439
after (16890, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-26.csv
before (10001, 2)
Min 1 Max 78 Avg 12.563743625637436
after (8318, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-27.csv
before (10001, 2)
Min 1 Max 44 Avg 12.495350464953505
after (8338, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-28.csv
before (6739, 2)
Min 1 Max 43 Avg 13.11989909482119
after (5270, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-29.csv
before (7313, 2)
Min 1 Max 38 Avg 12.91891152741693
after (6116, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-04-30.csv
before (10001, 2)
Min 1 Max 35 Avg 12.868713128687132
after (8633, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-05-01.csv
before (15001, 2)
Min 1 Max 62 Avg 12.321845210319312
after (12277, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-05-02.csv
before (15001, 2)
Min 1 Max 38 Avg 11.969068728751417
after (12447, 2)
NepQuake15/RawNepQuake15/Nepali-Tweets-05-03.csv
b

In [66]:
agg = agg.sort_values(by=['Date'])
agg['Date'].value_counts(), agg.shape

(Date
 2015-04-25    16890
 2015-05-18    13121
 2015-05-20    13091
 2015-05-13    12989
 2015-05-21    12988
 2015-05-14    12966
 2015-05-17    12952
 2015-05-15    12925
 2015-05-11    12893
 2015-05-19    12887
 2015-05-16    12871
 2015-05-12    12817
 2015-05-04    12614
 2015-05-02    12447
 2015-05-03    12409
 2015-05-01    12277
 2015-04-30     8633
 2015-04-27     8338
 2015-04-26     8318
 2015-04-29     6116
 2015-11-30     5856
 2015-04-28     5270
 2015-12-29     4177
 Name: count, dtype: int64,
 (255845, 2))

In [61]:
agg.to_csv("NepQuake15/finalized.csv",index=False)

In [64]:
test = agg[agg.apply(lambda x: len(x['Text'].split(" "))>=10,axis=1)]
test = test.sort_values(by=['Date'])
test['Date'].value_counts(), test.shape

(Date
 2015-04-25    10454
 2015-05-01     8451
 2015-05-02     8256
 2015-05-14     8247
 2015-05-12     8189
 2015-05-13     8141
 2015-05-15     8139
 2015-05-18     8112
 2015-05-04     8084
 2015-05-03     8083
 2015-05-20     8044
 2015-05-21     8003
 2015-05-17     7973
 2015-05-16     7963
 2015-05-11     7943
 2015-05-19     7745
 2015-04-30     6121
 2015-04-26     5779
 2015-04-27     5742
 2015-04-29     4337
 2015-04-28     3791
 2015-11-30     3687
 2015-12-29     2583
 Name: count, dtype: int64,
 (163867, 2))

In [ ]:
test.to_csv("NepQuake15/finalized.csv",index=False)